# Day 41: Plan Verification System

In this lab, we implement a **Plan Verification System**.
Modern agents often generate a multi-step plan before execution. This system inspects that plan for:
1.  **Forbidden Tools**: Attempting to use tools found on a blocklist.
2.  **Unsafe Sequences**: Performing dangerous actions (like deletion) without necessary prerequisites (like backup).

In [ ]:
import sys
import os

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.agents.plan_verifier import PlanVerifier, PlanStep

## 1. Clean Plan (Safe)

This plan performs a backup before deletion. It should pass.

In [ ]:
verifier = PlanVerifier()

safe_plan = [
    PlanStep(step_id=1, tool_name="list_files", arguments={}),
    PlanStep(step_id=2, tool_name="backup_file", arguments={"file": "important.txt"}),
    PlanStep(step_id=3, tool_name="delete_file", arguments={"file": "important.txt"})
]

result = verifier.verify_plan(safe_plan)

if result.is_valid:
    print("✅ Plan Validated.")
else:
    print("❌ Plan Rejected:")
    for err in result.errors:
        print(f"  - {err}")

## 2. Unsafe Sequence (Logic Violation)

This plan attempts to delete a file WITHOUT backing it up first. It should be rejected.

In [ ]:
unsafe_plan = [
    PlanStep(step_id=1, tool_name="list_files", arguments={}),
    PlanStep(step_id=2, tool_name="delete_file", arguments={"file": "data.csv"}) # Missing backup!
]

result = verifier.verify_plan(unsafe_plan)

if result.is_valid:
    print("✅ Plan Validated.")
else:
    print("❌ Plan Rejected:")
    for err in result.errors:
        print(f"  - {err}")

## 3. Restricted Tool

This plan tries to use a forbidden tool (`system_shell`).

In [ ]:
forbidden_plan = [
    PlanStep(step_id=1, tool_name="system_shell", arguments={"cmd": "rm -rf /"})
]

result = verifier.verify_plan(forbidden_plan)

if result.is_valid:
    print("✅ Plan Validated.")
else:
    print("❌ Plan Rejected:")
    for err in result.errors:
        print(f"  - {err}")